### Résolution de Sudoku avec Z3

Dans ce notebook, nous allons explorer comment utiliser Z3, un puissant SMT solver développé par Microsoft, pour résoudre des puzzles de Sudoku. Nous commencerons par une implémentation simple en utilisant des entiers, puis nous introduirons des déclinaisons plus sophistiquées.

#### 1. Introduction à Z3

Z3 est un SMT solver (Satisfiability Modulo Theories) qui peut être utilisé pour résoudre des problèmes de logique du premier ordre. Il permet de vérifier la satisfiabilité d'une formule logique sous certaines contraintes et est particulièrement efficace pour résoudre des problèmes impliquant des contraintes complexes, comme les puzzles de Sudoku.

Un SMT solver combine des techniques de résolution SAT (Satisfiability) avec des théories comme l'arithmétique, les tableaux, les bit-vectors, etc., permettant ainsi de résoudre des problèmes logiques beaucoup plus riches.

**Références :**
- [Exemples en C#](https://github.com/Z3Prover/z3/blob/master/examples/dotnet/Program.cs)
- [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html) dans le langage Z3 dédié SLib.
- [Z3Py Guide Examples](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) en Python
- [Online Z3 guide](https://microsoft.github.io/z3guide/) en js, SLib, Python

#### 2. Configuration de l'environnement

Nous allons installer le package Z3. Assurez-vous que votre environnement est correctement configuré pour exécuter du code .NET interactif.


In [1]:
#r "nuget: Microsoft.Z3"

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [2]:
#!import Sudoku-0-Environment.ipynb

#### 3. Implémentation de base avec des entiers

Comme nous allons tester plusieurs stratégies de résolution, nous commencerons par implémenter un solver simple utilisant des entiers pour représenter les cellules du Sudoku et fournissant les méthodes pour construire les contraintes.


In [3]:
// Importer les bibliothèques nécessaires
using System.Diagnostics;
using System;
using System.Collections;
using System.Collections.Generic;
using Microsoft.Z3;

// Classe pour résoudre le Sudoku en utilisant Z3 avec des entiers
public class Z3IntSolverSimple : ISudokuSolver
{
    public static Context ctx = new Context();
    public static BoolExpr _GenericContraints;
    public static IntExpr[][] CellVariables = new IntExpr[9][];
    public Z3IntSolverSimple()
    {
        // Initialiser les variables de cellule
        for (uint i = 0; i < 9; i++)
        {
            CellVariables[i] = new IntExpr[9];
            for (uint j = 0; j < 9; j++)
                CellVariables[i][j] = (IntExpr)ctx.MkConst(ctx.MkSymbol("x_" + (i + 1) + "_" + (j + 1)), ctx.IntSort);
        }
    }

    // Contraintes génériques pour tous les Sudokus, conservées en mémoire pour éviter de les recalculer
    public static BoolExpr GenericContraints
    {
        get
        {
            if (_GenericContraints == null)
            {
                _GenericContraints = GetGenericConstraints();
            }
            return _GenericContraints;
        }
    }
    
    // Générer les contraintes génériques
    public static BoolExpr GetGenericConstraints()
    {
        // Chaque cellule contient une valeur entre 1 et 9
        Expr[][] cells_c = new Expr[9][];
        for (uint i = 0; i < 9; i++)
        {
            cells_c[i] = new BoolExpr[9];
            for (uint j = 0; j < 9; j++)
                cells_c[i][j] = ctx.MkAnd(ctx.MkLe(ctx.MkInt(1), CellVariables[i][j]),
                                          ctx.MkLe(CellVariables[i][j], ctx.MkInt(9)));
        }
        // Chaque ligne contient des chiffres distincts
        BoolExpr[] rows_c = new BoolExpr[9];
        for (uint i = 0; i < 9; i++)
            rows_c[i] = ctx.MkDistinct(CellVariables[i]);
        // Chaque colonne contient des chiffres distincts
        BoolExpr[] cols_c = new BoolExpr[9];
        for (uint j = 0; j < 9; j++)
        {
            IntExpr[] column = new IntExpr[9];
            for (uint i = 0; i < 9; i++)
                column[i] = CellVariables[i][j];
            cols_c[j] = ctx.MkDistinct(column);
        }
        // Chaque carré 3x3 contient des chiffres distincts
        BoolExpr[][] sq_c = new BoolExpr[3][];
        for (uint i0 = 0; i0 < 3; i0++)
        {
            sq_c[i0] = new BoolExpr[3];
            for (uint j0 = 0; j0 < 3; j0++)
            {
                IntExpr[] square = new IntExpr[9];
                for (uint i = 0; i < 3; i++)
                    for (uint j = 0; j < 3; j++)
                        square[3 * i + j] = CellVariables[3 * i0 + i][3 * j0 + j];
                sq_c[i0][j0] = ctx.MkDistinct(square);
            }
        }
        BoolExpr sudoku_c = ctx.MkTrue();
        foreach (BoolExpr[] t in cells_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(rows_c), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(cols_c), sudoku_c);
        foreach (BoolExpr[] t in sq_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);
        return sudoku_c;
    }

    // Générer les contraintes spécifiques à une grille de Sudoku donnée
    public BoolExpr GetPuzzleConstraints(SudokuGrid grid)
    {
        BoolExpr instance_c = ctx.MkTrue();
        for (uint i = 0; i < 9; i++)
            for (uint j = 0; j < 9; j++)
                if (grid.Cells[i,j] != 0)
                {
                    instance_c = ctx.MkAnd(instance_c,
                        (BoolExpr)ctx.MkEq(CellVariables[i][j], ctx.MkInt(grid.Cells[i,j])));
                }
        return instance_c;
    }

    // Résoudre le Sudoku
    public SudokuGrid Solve(SudokuGrid grid)
    {
        SudokuGrid solution = new SudokuGrid();
        var sudoku_c = GenericContraints;
        var instance_c = GetPuzzleConstraints(grid);
        Solver s = ctx.MkSolver();
        s.Assert(sudoku_c);
        s.Assert(instance_c);        
        if (s.Check() == Status.SATISFIABLE)
        {
            Model m = s.Model;
            for (uint i = 0; i < 9; i++)
            {
                for (uint j = 0; j < 9; j++)
                {
                    solution.Cells[i,j] = ((IntNum)m.Evaluate(CellVariables[i][j])).Int;
                }
            }
        }
        else
        {
            Console.WriteLine("Failed to solve sudoku");
            throw new Exception("Failed to solve sudoku");
        }
        return solution;
    }
}

### 7. Utilisation de vecteurs de bits

Nous allons maintenant explorer une autre piste d'amélioration en utilisant des vecteurs de bits pour représenter les cellules du Sudoku. Cette approche peut réduire la taille des variables et améliorer les performances du solver.

Comme nous testerons plusieurs types de résolution, nous introduisons une classe de base qui fournit les contraintes.

In [4]:
using Microsoft.Z3;

public abstract class Z3BitVectorSolverBase : ISudokuSolver
{
    public static Context ctx = new Context();
    public static BoolExpr _GenericContraints;
    public static BitVecExpr[][] CellVariables = new BitVecExpr[9][];

    private static Sort BitVectorSort = ctx.MkBitVecSort(4);

    public static Solver _ReusableSolver;

    public Z3BitVectorSolverBase()
    {
        // Initialiser les variables de cellule en tant que vecteurs de 4 bits
        for (uint i = 0; i < 9; i++)
        {
            CellVariables[i] = new BitVecExpr[9];
            for (uint j = 0; j < 9; j++)
                CellVariables[i][j] = (BitVecExpr)ctx.MkConst(ctx.MkSymbol("x_" + (i + 1) + "_" + (j + 1)), BitVectorSort);
        }
    }

    // Contraintes génériques pour le Sudoku
    public static BoolExpr GenericContraints
    {
        get
        {
            if (_GenericContraints == null)
            {
                _GenericContraints = GetGenericConstraints();
            }
            return _GenericContraints;
        }
    }

    public static Solver ReusableSolver
    {
        get
        {
            if (_ReusableSolver == null)
            {
                _ReusableSolver = MakeReusableSolver();
            }
            return _ReusableSolver;
        }
    }

    public static Solver MakeReusableSolver()
    {
        Solver s = ctx.MkSolver();
        s.Assert(GenericContraints);
        return s;
    }

    // Obtenir l'expression constante pour une valeur donnée
    protected static BitVecExpr GetConstExpr(int value)
    {
        return (BitVecExpr)ctx.MkNumeral(value, BitVectorSort);
    }

    // Générer les contraintes génériques pour les vecteurs de bits
    public static BoolExpr GetGenericConstraints()
    {
        // Chaque cellule contient une valeur entre 1 et 9
        Expr[][] cells_c = new Expr[9][];
        for (uint i = 0; i < 9; i++)
        {
            cells_c[i] = new BoolExpr[9];
            for (uint j = 0; j < 9; j++)
                cells_c[i][j] = ctx.MkAnd(ctx.MkBVULE(GetConstExpr(1), CellVariables[i][j]),
                    ctx.MkBVULE(CellVariables[i][j], GetConstExpr(9)));
        }

        // Chaque ligne contient des chiffres distincts
        BoolExpr[] rows_c = new BoolExpr[9];
        for (uint i = 0; i < 9; i++)
            rows_c[i] = ctx.MkDistinct(CellVariables[i]);

        // Chaque colonne contient des chiffres distincts
        BoolExpr[] cols_c = new BoolExpr[9];
        for (uint j = 0; j < 9; j++)
        {
            BitVecExpr[] column = new BitVecExpr[9];
            for (uint i = 0; i < 9; i++)
                column[i] = CellVariables[i][j];

            cols_c[j] = ctx.MkDistinct(column);
        }

        // Chaque carré 3x3 contient des chiffres distinct
        BoolExpr[][] sq_c = new BoolExpr[3][];
        for (uint i0 = 0; i0 < 3; i0++)
        {
            sq_c[i0] = new BoolExpr[3];
            for (uint j0 = 0; j0 < 3; j0++)
            {
                BitVecExpr[] square = new BitVecExpr[9];
                for (uint i = 0; i < 3; i++)
                for (uint j = 0; j < 3; j++)
                    square[3 * i + j] = CellVariables[3 * i0 + i][3 * j0 + j];
                sq_c[i0][j0] = ctx.MkDistinct(square);
            }
        }

        BoolExpr sudoku_c = ctx.MkTrue();
        foreach (BoolExpr[] t in cells_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(rows_c), sudoku_c);
        sudoku_c = ctx.MkAnd(ctx.MkAnd(cols_c), sudoku_c);
        foreach (BoolExpr[] t in sq_c)
            sudoku_c = ctx.MkAnd(ctx.MkAnd(t), sudoku_c);

        return sudoku_c;
    }

    // Générer les contraintes spécifiques à une grille de Sudoku donnée
    public BoolExpr GetPuzzleConstraints(SudokuGrid grid)
    {
        BoolExpr instance_c = ctx.MkTrue();
        for (uint i = 0; i < 9; i++)
        for (uint j = 0; j < 9; j++)
            if (grid.Cells[i,j] != 0)
            {
                instance_c = ctx.MkAnd(instance_c,
                    (BoolExpr)ctx.MkEq(CellVariables[i][j], GetConstExpr(grid.Cells[i,j])));
            }

        return instance_c;
    }

    // Méthode abstraite pour résoudre le Sudoku
    public abstract SudokuGrid Solve(SudokuGrid s);
}


### 4. Implémentation simple avec vecteurs de bits

Nous allons implémenter un solver simple en utilisant des vecteurs de bits.

In [5]:
using Microsoft.Z3;

public class Z3BitVectorSolverSimple : Z3BitVectorSolverBase
{
    public override SudokuGrid Solve(SudokuGrid s)
    {
        SudokuGrid solution = new SudokuGrid();
        SudokuSolve(s, ref solution);
        return solution;
    }

    public void SudokuSolve(SudokuGrid grid, ref SudokuGrid solution)
    {
        var sudoku_c = GenericContraints;
        var instance_c = GetPuzzleConstraints(grid);
        Solver s = ctx.MkSolver();
        s.Assert(sudoku_c);
        s.Assert(instance_c);

        if (s.Check() == Status.SATISFIABLE)
        {
            Model m = s.Model;
            for (uint i = 0; i < 9; i++)
            {
                for (uint j = 0; j < 9; j++)
                {
                    solution.Cells[i,j] = ((BitVecNum)m.Evaluate(CellVariables[i][j])).Int;
                }
            }
        }
        else
        {
            Console.WriteLine("Failed to solve sudoku");
            throw new Exception("Failed to solve sudoku");
        }
    }
}


### 5. Utilisation de l'API de substitution avec vecteurs de bits

Nous allons implémenter une classe utilisant l'API de substitution. Cette approche peut améliorer les performances en réutilisant les contraintes génériques et en substituant uniquement les valeurs spécifiques à la grille de Sudoku en cours de résolution.


In [6]:
using Microsoft.Z3;


public class Z3BitVectorSolverSubstitution : Z3BitVectorSolverBase
{
    public override SudokuGrid Solve(SudokuGrid s)
    {
        SudokuGrid solution = new SudokuGrid();
        SudokuSolve(s, ref solution);
        return solution;
    }

    public void SudokuSolve(SudokuGrid grid, ref SudokuGrid solution)
    {
        var substExprs = new List<Expr>();
        var substVals = new List<Expr>();

        for (int i = 0; i < 9; i++)
        for (int j = 0; j < 9; j++)
            if (grid.Cells[i,j] != 0)
            {
                substExprs.Add(CellVariables[i][j]);
                substVals.Add(GetConstExpr(grid.Cells[i,j]));
            }

        // Utiliser l'API de substitution pour appliquer les contraintes spécifiques
        BoolExpr instance_c = (BoolExpr)GenericContraints.Substitute(substExprs.ToArray(), substVals.ToArray());
        Solver solver = ctx.MkSolver();
        solver.Assert(instance_c);
        if (solver.Check() == Status.SATISFIABLE)
        {
            Model m = solver.Model;
            for (uint i = 0; i < 9; i++)
            {
                for (uint j = 0; j < 9; j++)
                {
                    if (grid.Cells[i,j] == 0)
                    {
                        solution.Cells[i,j] = ((BitVecNum)m.Evaluate(CellVariables[i][j])).Int;
                    }
                    else
                    {
                        solution.Cells[i,j] = grid.Cells[i,j];
                    }
                }
            }
        }
        else
        {
            Console.WriteLine("Failed to solve sudoku");
            throw new Exception("Failed to solve sudoku");
        }
    }
}


### 6. Utilisation de l'API de tactiques avec vecteurs de bits

Nous allons maintenant explorer l'utilisation de l'API de tactiques de Z3 pour résoudre les Sudokus. Les tactiques permettent d'appliquer des transformations et des simplifications spécifiques pour améliorer l'efficacité de la résolution.

In [7]:
using Microsoft.Z3;
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

internal class Z3BitSub_Tactic : Z3BitVectorSolverBase
{
    public override SudokuGrid Solve(SudokuGrid s)
    {
        SudokuGrid solution = new SudokuGrid();
        SudokuSolve(s, ref solution);
        return solution;
    }
    public void SudokuSolve(SudokuGrid grid, ref SudokuGrid solution)
    {
        var substExprs = new List<Expr>();
        var substVals = new List<Expr>();
        for (int i = 0; i < 9; i++)
            for (int j = 0; j < 9; j++)
                if (grid.Cells[i,j] != 0)
                {
                    substExprs.Add(CellVariables[i][j]);
                    substVals.Add(GetConstExpr(grid.Cells[i,j]));
                }
        BoolExpr instance_c = (BoolExpr)GenericContraints.Substitute(substExprs.ToArray(), substVals.ToArray());
        Solver solver = ctx.MkSolver();
        solver.Assert(instance_c);
        BoolExpr puzzleConstraints = GetPuzzleConstraints(grid);
        
        // Utiliser l'API de tactiques pour simplifier et résoudre le Sudoku
        Tactic tactic = ctx.MkTactic("simplify");
        Goal goal = ctx.MkGoal();
        goal.Assert(ctx.MkAnd(GenericContraints, puzzleConstraints));
        ApplyResult applyResult = tactic.Apply(goal);
        if (applyResult.NumSubgoals > 0)
        {
            Goal newGoal = applyResult.Subgoals[0];
            solver.Assert(newGoal.Formulas);
            if (solver.Check() == Status.SATISFIABLE)
            {
                Model m = solver.Model;
                for (uint i = 0; i < 9; i++)
                {
                    for (uint j = 0; j < 9; j++)
                    {
                        if (grid.Cells[i,j] == 0)
                        {
                            solution.Cells[i,j] = ((BitVecNum)m.Evaluate(CellVariables[i][j])).Int;
                        }
                        else
                        {
                            solution.Cells[i,j] = grid.Cells[i,j];
                        }
                    }
                }
            }
            else
            {
                Console.WriteLine("Failed to solve sudoku");
                throw new Exception("Failed to solve sudoku");
            }
        }
    }
}


### 7. Comparaison des solveurs

Pour comparer les différentes implémentations de solveurs, nous utiliserons une approche similaire à celle de notre notebook OR-Tools. Nous évaluerons les performances de chaque solver sur des puzzles de Sudoku de différentes difficultés (Facile, Moyen, Difficile).

In [8]:
var solvers = new List<(string Name, ISudokuSolver Solver)>
{
    ("Z3 Int Solver Simple", new Z3IntSolverSimple()),
    ("Z3 Bit Vector Solver Simple", new Z3BitVectorSolverSimple()),
    ("Z3 Bit Vector Solver Substitution", new Z3BitVectorSolverSubstitution()),
    ("Z3 Bit Vector Solver Tactic", new Z3BitSub_Tactic())
};

var results = SudokuHelper.TestSolvers(solvers);
SudokuHelper.DisplayResults(results);



### Conclusion

Dans ce notebook, nous avons exploré différentes approches pour résoudre des puzzles de Sudoku en utilisant Z3. Nous avons commencé par une implémentation simple en utilisant des entiers, puis nous avons introduit des méthodes plus sophistiquées utilisant des vecteurs de bits et l'API de substitution. Nous avons également comparé les performances de ces différentes approches.

Les résultats montrent que les solveurs utilisant des vecteurs de bits et l'API de substitution offrent des performances améliorées par rapport à l'approche simple basée sur les entiers. L'utilisation des tactiques de simplification peut également apporter des gains de performance significatifs mais nécessite plus de tests.

Merci d'avoir suivi ce notebook, et j'espère que cela vous a permis de mieux comprendre comment utiliser Z3 pour résoudre des problèmes de contraintes complexes comme les puzzles de Sudoku.